In [13]:
import pandas as pd
import numpy as np
import time
from neo4j import GraphDatabase

In [5]:
df_pois = pd.read_csv('Datasets/pois_final.csv')

In [3]:
df_pois.head()

,longitude,latitude,type,class,name
0,-70.566756,-33.411691,Sports Centre,sport_and_leisure,Pacific Fitness
1,-70.568423,-33.411289,Hospital,medical,Cirujias Plasticas Acqua
2,-70.566457,-33.412217,Clinic,medical,Clínica CELA Manquehue
3,-70.568498,-33.411080,Hospital,medical,NovaVida
4,-70.568729,-33.411113,Kindergarten,education_prim,Sol del Inca


In [6]:
# Diccionario para mapear cada tipo de POI a su categoría (1, 2 o 3)
poi_category_mapping = {
    # Categoría 1 (600m)
    'School': 1, 'Kindergarten': 1,
    'Restaurant': 1, 'Fast Food': 1, 'Cafe': 1, 'Bar': 1, 'Pub': 1, 'Ice Cream': 1,
    'Convenience': 1, 'Alcohol': 1, 'Bakery': 1, 'Butcher': 1, 'Greengrocer': 1,
    'Pastry': 1, 'Confectionery': 1, 'Seafood': 1, 'Beverages': 1, 'Wine': 1,
    'Deli': 1, 'Farm': 1, 'Grocery': 1,
    'Pharmacy': 1, 'Dentist': 1, 'Doctors': 1,
    'Park': 1, 'Playground': 1, 'Recreation Ground': 1, 'Garden': 1, 'Dog Park': 1,
    'Picnic Site': 1, 'Village Green': 1, 'Wood': 1,
    'Place Of Worship': 1, 'Church': 1, 'Religious': 1, 'Chapel': 1, 'Temple': 1,
    'Pitch': 1, 'Fitness Station': 1, 'Outdoor Seating': 1, 'Churchyard': 1,

    # Categoría 2 (1200m)
    'Library': 2, 'Track': 2, 'Amusement Arcade': 2,
    'Nightclub': 2, 'Food Court': 2, 'Brewery': 2, 'Biergarten': 2,
    'Supermarket': 2, 'Marketplace': 2, 'Market': 2,
    'Hospital': 2, # Ajuste: Hospital a Cat 2 (1200m)
    'Healthcare': 2, # Mantener Healthcare en Cat 2, similar a Hospital en alcance.
    'Cemetery': 2, 'Nature Reserve': 2, 'Grave Yard': 2,
    'Monastery': 2,
    'Swimming Pool': 2, 'Sports Centre': 2, 'Fitness Centre': 2, 'Fitness': 2,
    'Sports Hall': 2, 'Dojo': 2, 'Dance': 2, 'Sport': 2, 'Bird Hide': 2, 'Yoga': 2,
    'Veterinary': 2,

    # Categoría 3 (2400m)
    'Museum': 3, 'Theatre': 3, 'Arts Centre': 3, 'Gallery': 3, 'Cinema': 3,
    'Attraction': 3, 'Water Park': 3, 'Casino': 3, 'Zoo': 3, 'Theme Park': 3,
    'Aquarium': 3, 'Raceway': 3,
    'University': 3, 'College': 3,
    'Clinic': 3, # Ajuste: Clinic a Cat 3 (2400m)
    'Camp Site': 3,
    'Police': 3, 'Fire Station': 3,
    'Golf Course': 3, 'Horse Riding': 3, 'Ice Rink': 3, 'Trampoline Park': 3,
}

# Crear la nueva columna 'cat' aplicando el mapeo
# Usamos .get(type, 0) para asignar 0 (o algún valor por defecto/NaN) si un tipo no se encuentra,
# aunque idealmente todos tus tipos deberían estar cubiertos.
df_pois['cat'] = df_pois['type'].map(poi_category_mapping).fillna(0).astype(int)

# Opcional: Verificar que no haya POIs sin categoría asignada (donde 'cat' sea 0)
if (df_pois['cat'] == 0).any():
    print("¡Advertencia! Algunos POIs no tienen una categoría asignada (cat = 0).")
    print("Tipos de POI sin categoría asignada:")
    print(df_pois[df_pois['cat'] == 0]['type'].unique())
    # Considera revisar el mapeo para estos tipos, o eliminarlos si no son relevantes.

# Opcional: Mostrar las primeras filas del DataFrame con la nueva columna
print(df_pois.head())

# Opcional: Contar la distribución de las categorías
print("\nConteo de POIs por categoría:")
print(df_pois['cat'].value_counts())

   longitude   latitude           type              class  \
0 -70.566756 -33.411691  Sports Centre  sport_and_leisure   
1 -70.568423 -33.411289       Hospital            medical   
2 -70.566457 -33.412217         Clinic            medical   
3 -70.568498 -33.411080       Hospital            medical   
4 -70.568729 -33.411113   Kindergarten     education_prim   

                       name  cat  
0           Pacific Fitness    2  
1  Cirujias Plasticas Acqua    2  
2    Clínica CELA Manquehue    3  
3                  NovaVida    2  
4              Sol del Inca    1  

Conteo de POIs por categoría:
cat
1    17267
2     2743
3     1434
Name: count, dtype: int64


In [9]:
df_pois.sample(10)

,longitude,latitude,type,class,name,cat,poi_id
20370,-70.767742,-33.495192,Pitch,sport_and_leisure,Cancha Batalla Maipú,1,20370
19957,-70.758739,-33.399778,Supermarket,food_and_drink_stores,Lider,2,19957
1793,-70.652548,-33.494483,Restaurant,food_and_drink,Pizzeria Bruce,1,1793
9459,-70.589012,-33.537175,School,education_prim,Colegio Almendral,1,9459
11893,-70.757609,-33.471143,Recreation Ground,park_like,Plaza Villa Mundo Nuevo,1,11893
20153,-70.605950,-33.582202,Supermarket,food_and_drink_stores,SuperBodega Acuenta,2,20153
16250,-70.705104,-33.405022,School,education_prim,Centro Educacional Federico Garcia Lorca,1,16250
13063,-70.701007,-33.589986,Bakery,food_and_drink_stores,NoName,1,13063
16638,-70.719301,-33.368470,Bakery,food_and_drink_stores,El Trigal,1,16638
2476,-70.609293,-33.453853,Clinic,medical,Consalud,3,2476


In [8]:
df_pois["poi_id"] = df_pois.index.astype('int64')
df_pois["name"] = df_pois["name"].fillna("NoName")

In [11]:
df_pois.to_csv('Datasets/pois_final.csv', index=False)

In [14]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [15]:
def crear_nodos_pois(tx, row):
    query = """
    MERGE (p:POI {
        poi_id: $poi_id
    })
    SET p.name = $name,
        p.latitude = $latitude,
        p.longitude = $longitude,
        p.class = $class,
        p.cat = $cat
    """
    tx.run(query, **row)

def cargar_pois(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "poi_id": row["poi_id"],
                "name": row["name"],
                "latitude": float(row["latitude"]),
                "longitude": float(row["longitude"]),
                "class": row["class"],
                "cat": row['cat']
            }
            session.write_transaction(crear_nodos_pois, datos)

In [16]:
start = time.time()
cargar_pois(df_pois)
end = time.time()
print(f'Tiempo de ejecucion = {end-start} segundos')

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_33708\3661416374.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodos_pois, datos)


Tiempo de ejecucion = 9156.431678533554 segundos
